In [1]:
# Viết code thực hiện mô hình Retriever Generator bằng các thư viện Pytorch
# Input: câu hỏi và 1 số file json chứa các đoạn văn
# Output: các đoạn văn được sắp xếp theo độ liên quan với câu hỏi
# Bước 1: Tạo các vector biểu diễn cho câu hỏi và các đoạn văn
# Bước 2: Tính độ tương đồng giữa câu hỏi và các đoạn văn
# Bước 3: Sắp xếp các đoạn văn theo độ tương đồng giảm dần
# Bước 4: Trả về kết quả



In [2]:
#Khai báo các thư viện để thực hiện các yêu cầu trên
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import json
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer



In [3]:
# Bước 1: Tạo các vector biểu diễn cho câu hỏi và các đoạn văn
# Hàm tạo vector biểu diễn cho câu hỏi
# Input: câu hỏi
# Output: vector biểu diễn cho câu hỏi
def create_question_vector(question):
    # Tách câu hỏi thành các từ
    question_words = word_tokenize(question)
    # Loại bỏ các từ không cần thiết
    question_words = remove_stopwords(question_words)
    # Chuẩn hóa các từ
    question_words = normalize_words(question_words)
    # Tạo vector biểu diễn cho câu hỏi
    question_vector = create_vector_from_words(question_words)
    return question_vector
# Hàm tạo vector biểu diễn cho các đoạn văn
# Input: đoạn văn
# Output: vector biểu diễn cho đoạn văn
def create_paragraph_vector(paragraph):
    # Tách đoạn văn thành các từ
    paragraph_words = word_tokenize(paragraph)
    # Loại bỏ các từ không cần thiết
    paragraph_words = remove_stopwords(paragraph_words)
    # Chuẩn hóa các từ
    paragraph_words = normalize_words(paragraph_words)
    # Tạo vector biểu diễn cho đoạn văn
    paragraph_vector = create_vector_from_words(paragraph_words)
    return paragraph_vector
# Hàm loại bỏ các từ không cần thiết
# Input: danh sách các từ
# Output: danh sách các từ sau khi loại bỏ các từ không cần thiết
def remove_stopwords(words):
    # Loại bỏ các từ không cần thiết
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    return words
# Hàm chuẩn hóa các từ
# Input: danh sách các từ
# Output: danh sách các từ sau khi chuẩn hóa
def normalize_words(words):
    # Chuẩn hóa các từ
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    return words
# Hàm tạo vector biểu diễn cho các từ
# Input: danh sách các từ
# Output: vector biểu diễn cho các từ
def create_vector_from_words(words):
    # Tạo vector biểu diễn cho các từ
    vectors = []
    for word in words:
        vector = get_vector_from_word(word)
        vectors.append(vector)
    return vectors

In [4]:
# Bước 2: Tính độ tương đồng giữa câu hỏi và các đoạn văn
# Hàm tính độ tương đồng giữa câu hỏi và các đoạn văn
# Input: câu hỏi, các đoạn văn
# Output: danh sách các đoạn văn được sắp xếp theo độ tương đồng giảm dần
def get_paragraphs_sorted_by_similarity(question, paragraphs):
    # Tạo vector biểu diễn cho câu hỏi
    question_vector = create_question_vector(question)
    # Tạo vector biểu diễn cho các đoạn văn
    paragraphs_vectors = []
    for paragraph in paragraphs:
        paragraph_vector = create_paragraph_vector(paragraph)
        paragraphs_vectors.append(paragraph_vector)
    # Tính độ tương đồng giữa câu hỏi và các đoạn văn
    similarities = []
    for paragraph_vector in paragraphs_vectors:
        similarity = get_similarity(question_vector, paragraph_vector)
        similarities.append(similarity)
    # Sắp xếp các đoạn văn theo độ tương đồng giảm dần
    paragraphs_sorted = [paragraph for _, paragraph in sorted(zip(similarities, paragraphs), reverse=True)]
    return paragraphs_sorted
# Hàm tính độ tương đồng giữa 2 vector
# Input: 2 vector
# Output: độ tương đồng giữa 2 vector
def get_similarity(vector1, vector2):
    # Tính độ tương đồng giữa 2 vector
    similarity = np.dot(vector1, vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))
    return similarity


In [5]:
# Bước 3: Sắp xếp các đoạn văn theo độ tương đồng giảm dần
# Hàm sắp xếp các đoạn văn theo độ tương đồng giảm dần
# Input: câu hỏi, các đoạn văn
# Output: danh sách các đoạn văn được sắp xếp theo độ tương đồng giảm dần
def get_paragraphs_sorted_by_similarity(question, paragraphs):
    # Tạo vector biểu diễn cho câu hỏi
    question_vector = create_question_vector(question)
    # Tạo vector biểu diễn cho các đoạn văn
    paragraphs_vectors = []
    for paragraph in paragraphs:
        paragraph_vector = create_paragraph_vector(paragraph)
        paragraphs_vectors.append(paragraph_vector)
    # Tính độ tương đồng giữa câu hỏi và các đoạn văn
    similarities = []
    for paragraph_vector in paragraphs_vectors:
        similarity = get_similarity(question_vector, paragraph_vector)
        similarities.append(similarity)
    # Sắp xếp các đoạn văn theo độ tương đồng giảm dần
    paragraphs_sorted = [paragraph for _, paragraph in sorted(zip(similarities, paragraphs), reverse=True)]
    return paragraphs_sorted


In [6]:
# Bước 4: Trả về kết quả
# Hàm trả về kết quả
# Input: câu hỏi, các đoạn văn
# Output: kết quả
def get_result(question, paragraphs):
    # Sắp xếp các đoạn văn theo độ tương đồng giảm dần
    paragraphs_sorted = get_paragraphs_sorted_by_similarity(question, paragraphs)
    # Trả về kết quả
    result = paragraphs_sorted[0]
    return result

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI20\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Hàm main và test
if __name__ == '__main__':
    # Đọc dữ liệu
    data = json.load(open('nttuan8_dl.json', encoding='utf-8'))
    # Lấy câu hỏi
    question = "Deep Learning là gì ?"
    # Lấy các đoạn văn từ 'content' của file json
    paragraphs = [row['content'] for row in data]
    # Trả về kết quả
    result = get_result(question, paragraphs)
    print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'nttuan8_dl.json'

In [ ]:
pip install -U farm-haystack


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from haystack.document_store.memory import InMemoryDocumentStore
from haystack.retriever.dense import DensePassageRetriever
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.pipeline import Pipeline

# Tạo InMemoryDocumentStore
document_store = InMemoryDocumentStore()

# Load các file json chứa các đoạn vănz
file_paths = ["file1.json", "file2.json", "file3.json"]
docs = convert_files_to_dicts(file_paths)

# Xóa các ký tự không cần thiết trong các đoạn văn
for doc in docs:
    doc["text"] = clean_wiki_text(doc["text"])

# Lưu các đoạn văn vào InMemoryDocumentStore
document_store.write_documents(docs)

# Tạo DensePassageRetriever để tìm kiếm các đoạn văn phù hợp với câu hỏi
retriever = DensePassageRetriever(document_store=document_store,
                                 query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                 passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                 use_gpu=False,
                                 embed_title=True,
                                 max_seq_len=256,
                                 batch_size=16)

# Tạo Pipeline để thực hiện các bước tiền xử lý và truy xuất
pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=retriever, name="Generator", inputs=["Retriever"])

# Định nghĩa câu hỏi và chạy Pipeline
question = "What is the capital of France?"
result = pipe.run(query=question)

# Lấy kết quả trả về
answers = result["Generator"]
for ans in answers:
    print(ans["text"])


ModuleNotFoundError: No module named 'haystack.document_store'

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModel

# load the json file
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# load the pretrained retriever model and tokenizer
model_name = "facebook/dpr-question_encoder-single-nq-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# define a function to retrieve relevant documents for
def retrieve_documents(question, k):
    # encode the question into tokens and convert to tensor
    question_input = tokenizer(question, return_tensors="pt", truncation=True, max_length=256)
    # get the embedding vector of the question from the model
    question_embed = model(**question_input).pooler_output # shape: (1, embed_size)

    # initialize an empty list to store the pairs of (similarity, document)
    results = []

    # loop through the elements in the json file
    for item in data:
        # get the content value as the document
        document = item.get("content")

        # encode the document into tokens and convert to tensor
        document_input = tokenizer(document, return_tensors="pt", truncation=True, max_length=256)

        # get the embedding vector of the document from the model
        document_embed = model(**document_input).pooler_output # shape: (1, embed_size)

        # compute the similarity between the question and document embeddings using dot product or cosine similarity
        similarity = torch.dot(question_embed.squeeze(), document_embed.squeeze()) # or torch.cosine_similarity(question_embed, document_embed)

        # append the pair of (similarity, document) to the results list
        results.append((similarity.item(), document))
    # sort the results list by descending order of similarity
    results.sort(key=lambda x: x[0], reverse=True)

    # get the top k pairs of (similarity, document)
    results = results[:k]

    # return the results list as the output
    return results

retrieve_documents("Giới thiệu về Deep learning ?", 5)


[(75.96726989746094,
  'Nội dung\nDeep learning là gì?\nÝ tưởng và mục đích của loạt bài viết này.\nSách Deep Learning cơ bản\nKhóa Deep Learning cơ bản\nNội dung của loạt bài viết\nMôi trường\nYêu cầu\nDeep learning là gì?\nTrong khoảng vài năm trở lại đây mọi người được nghe rất nhiều về cách mạng công nghiệp 4.0. Vậy điều gì đã làm lên sự đặc biệt? Hạt nhân cho sự bùng nổ đến từ Artificial Intelligence (trí tuệ nhân tạo) hay cụ thể là Machine Learning (máy học). Ứng dụng của AI thì ở khắp mọi nơi như là: ô tô tự lái; robot phẫu thuật; hệ thống dịch tự động; chatbot tự động trả lời; AlphaGo,…\nMachine learning theo định nghĩa của wiki, “Machine learning is the subfield of computer science that “gives computers the ability to learn without being explicitly programmed “, ý là chương trình có thể tự học được từ dữ liệu.\nMachine learning là một ngành rất rộng và nặng về toán, gồm rất nhiều thuật toán và mỗi thuật toán có ứng dụng riêng tùy vào bài toán:\nLinear Regression\nLogistic Regr